# Configuring Rocoto workflows

The `uw rocoto` tool has three modes:

* `realize` -- creates a Rocoto XML from a UW YAML
* `validate` -- uses the UW framework to validate a Rocoto XML without needing to have a Rocoto installation
* `iterate` -- runs a Rocoto XML to completion through a specified task

For examples of how to use these CLI tools, go to the __[Rocoto UW docs](https://uwtools.readthedocs.io/en/main/sections/user_guide/cli/tools/rocoto.html)__.

# Building a UW YAML for Rocoto

We'll start by building a UW YAML config describing the Rocoto workflow that will run the cycling experiment's tasks. Use the __[UW YAML Rocoto Workflows](https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/rocoto.html)__ docs to get going.

In [ ]:
import os
from pathlib import Path

# IN BASH: export configs=/path/to/uwtools_training/configs
os.environ["configs"] = str(Path(".").resolve().parent / "configs")

## The workflow section

In [ ]:
!head -n 20 $configs/rocoto_workflow.yaml 

## Adding a task

Docs: __[Defining Tasks](https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/rocoto.html#defining-tasks)__

* The UW YAML key should be `task_` followed by an arbitrary name. This name will be shown in `rocotostat` output.
* Values can be string references to XML entities, Jinja2 expressions, hard-coded values.
* Don't use the `--batch` flag when running UW drivers: Rocoto handles submitting jobs to the batch system.
* Use Rocoto-supplied information for date-time flexibility.
* A `jobname` variable, reflecting the arbitrary name following `task_`, will be supplied by `uwtools` for use in Jinja2 expressions.


# Check the progress

Run the `uw rocoto realize` tool to validate the YAML and attempt to create an XML.

In [ ]:
!uw rocoto realize -c $configs/rocoto_workflow.yaml -o workflow.xml

## OOPS!
The `rocoto_workflow.yaml` doesn't have all the values it needs. Let's compose a more complete YAML on the fly.

In [ ]:
%%bash
realize_args=(
    -i $configs/prepare_bcs.yaml
    -u $configs/rocoto_one_task.yaml
)
    
uw config realize ${realize_args[*]} | uw rocoto realize -o workflow.xml
cat workflow.xml

## Adding a metatask

Docs: __[Defining Metatasks](https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/rocoto.html#defining-metatasks)__

* Metatask blocks require a `var:` block and one more `task_` or `metatask_` blocks.
* The `var:` block has at least one key/value pair to define the iteration loop.

In [ ]:
%%bash
realize_args=(
    -i $configs/prepare_bcs.yaml
    -u $configs/rocoto_with_metatask.yaml   
)

uw config realize ${realize_args[*]} | uw rocoto realize -o workflow.xml

In [ ]:
!cat workflow.xml

## Adding dependencies to a task

Docs: __[The dependency: key](https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/rocoto.html#the-dependency-key)__

* UW Rocoto YAML follows the same structure as Rocoto XML
* YAML keys must be unique at the same level, so all *dependencies* take an optional suffix after an underscore, e.g., `_arbitrary`, where `arbitrary` can be any unique string identifier. This identifier is not used by Rocoto, but is used when referenced by Jinja2 expressions. While they are not required, they can also be useful to make some dependencies clearer and more human-readable.

In [ ]:
%%bash 
uw config realize -i $configs/fv3_config.yaml -u $configs/gsi_config.yaml | \
    uw config realize -u $configs/prepare_bcs.yaml | \
    uw config realize -u $configs/rocoto_workflow.yaml | \
    uw rocoto realize -o workflow.xml

In [ ]:
!cat workflow.xml

## Use `compose` tool 

* A helper tool that reduces the need for repeated piping.
* Configs increase in priority from left to right. Last one takes most precedence.

In [ ]:
%%bash
file_list=(
    $configs/fv3_config.yaml
    $configs/gsi_config.yaml
    $configs/prepare_bcs.yaml
    $configs/rocoto_workflow.yaml
)
uw config compose ${file_list[*]} -o experiment.yaml

In [ ]:
!cat experiment.yaml

### `compose` and `realize`

In [ ]:
%%bash
file_list=(
    $configs/fv3_config.yaml
    $configs/gsi_config.yaml
    $configs/prepare_bcs.yaml
    $configs/rocoto_workflow.yaml
)
uw config compose ${file_list[*]} -o experiment.yaml --realize

In [ ]:
!cat experiment.yaml

## Visually inspect the Rocoto XML

* Check to make sure there are no remaining Jinja2 expressions that got past the validator.
* Did any loops end up off by one?
* Anything else amiss?


## Run the Rocoto Workflow

Docs: __[uw rocoto iterate](https://uwtools.readthedocs.io/en/main/sections/user_guide/cli/tools/rocoto.html#iterate)__

The `uw rocoto iterate` tool iteratively runs `rocotorun` and `rocotostat` in subprocesses at a default interval of 10 s to step through the workflow until a particular cycle and task are complete.

In [ ]:
%%bash
module load rocoto
uw rocoto iterate -w workflow.xml -d workflow.db --cycle 2025-08-27T13 --task cycled_forecast